In [1]:
import notebook_utils
from birddog.utility import ARCHIVE_BASE, ARCHIVE_LIST, find_subarchives, translate_page, get_text 
from birddog.core import Archive
#from bs4 import BeautifulSoup
#import requests
import json

current working dir: /Users/jbrandt/code/birddog
Using AWS S3 bucket birddog-data for storage.


In [2]:
archives = {}
for key, value in ARCHIVE_LIST.items():
    print(key)
    archives[key] = find_subarchives(value)

DAARC
DACHGO
DACHkO
DADNO
DAHO
DAHeO
DAHmO
DAIFO
DAK
DAKO
DAKrO
DALO
DALuO
DAMO
DAOO
DAPO
DARO
DAS
DASO
DAVIO
DAVOO
DAZHO
DAZKO
DAZpO


In [3]:
print(json.dumps(archives, indent=4))

{
    "DAARC": {
        "\u0414": {
            "title": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0410\u0420\u041a/\u0414"
            },
            "archive": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0410\u0420\u041a"
            },
            "subarchive": {
                "uk": "\u0414"
            },
            "description": {
                "uk": "\u0424\u043e\u043d\u0434\u0438 \u0434\u043e\u0440\u0430\u0434\u044f\u043d\u0441\u044c\u043a\u043e\u0433\u043e \u043f\u0435\u0440\u0456\u043e\u0434\u0443"
            },
            "link": "/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%90%D0%A0%D0%9A/%D0%94"
        }
    },
    "DACHGO": {
        "\u0414": {
            "title": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0427\u0433\u041e/\u0414"
            },
            "archive": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0427\u0433\u041e"
            },
    

In [4]:
translate_page(archives, dry_run=True)

220

In [5]:
#translate_page(archives)

In [6]:
print(json.dumps(archives, indent=4))

{
    "DAARC": {
        "\u0414": {
            "title": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0410\u0420\u041a/\u0414"
            },
            "archive": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0410\u0420\u041a"
            },
            "subarchive": {
                "uk": "\u0414"
            },
            "description": {
                "uk": "\u0424\u043e\u043d\u0434\u0438 \u0434\u043e\u0440\u0430\u0434\u044f\u043d\u0441\u044c\u043a\u043e\u0433\u043e \u043f\u0435\u0440\u0456\u043e\u0434\u0443"
            },
            "link": "/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%90%D0%A0%D0%9A/%D0%94"
        }
    },
    "DACHGO": {
        "\u0414": {
            "title": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0427\u0433\u041e/\u0414"
            },
            "archive": {
                "uk": "\u0410\u0440\u0445\u0456\u0432:\u0414\u0410\u0427\u0433\u041e"
            },
    

In [7]:
archives

{'DAARC': {'Д': {'title': {'uk': 'Архів:ДААРК/Д'},
   'archive': {'uk': 'Архів:ДААРК'},
   'subarchive': {'uk': 'Д'},
   'description': {'uk': 'Фонди дорадянського періоду'},
   'link': '/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%90%D0%A0%D0%9A/%D0%94'}},
 'DACHGO': {'Д': {'title': {'uk': 'Архів:ДАЧгО/Д'},
   'archive': {'uk': 'Архів:ДАЧгО'},
   'subarchive': {'uk': 'Д'},
   'description': {'uk': 'Д — Фонди дорадянського періоду'},
   'link': '/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%A7%D0%B3%D0%9E/%D0%94'},
  'Р': {'title': {'uk': 'Архів:ДАЧгО/Р'},
   'archive': {'uk': 'Архів:ДАЧгО'},
   'subarchive': {'uk': 'Р'},
   'description': {'uk': 'Р — Фонди періоду після 1917 року'},
   'link': '/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%A7%D0%B3%D0%9E/%D0%A0'}},
 'DACHkO': {'Д': {'title': {'uk': 'Архів:ДАЧкО/Д'},
   'archive': {'uk': 'Архів:ДАЧкО'},
   'subarchive': {'uk': 'Д'},
   'description': {'uk': 'Фонди дорадянського періоду'},
   'link': '/wiki/%D0%90%